<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/GRU-intra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import GRU, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [5]:
y_data = data.pop('4')
x_data = data

In [6]:
train_x,test_x,train_y,test_y = train_test_split(x_data, y_data, test_size=0.2, random_state=0, shuffle=True)

In [7]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [8]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [9]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [10]:
model = Sequential()
model.add(GRU(100,input_shape=(len(x_data.columns), 1), activation='tanh', recurrent_activation='sigmoid'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 100)               30900     
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 31,304
Trainable params: 31,304
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(train_x, train_y, epochs=6, batch_size=32)

Epoch 1/6
414237/414237 [==============================] - 2005s 5ms/step - loss: 0.0395 - accuracy: 0.9865
Epoch 2/6
414237/414237 [==============================] - 2021s 5ms/step - loss: 0.0270 - accuracy: 0.9908
Epoch 3/6
414237/414237 [==============================] - 1972s 5ms/step - loss: 0.0250 - accuracy: 0.9913
Epoch 4/6
414237/414237 [==============================] - 1949s 5ms/step - loss: 0.0240 - accuracy: 0.9915
Epoch 5/6
414237/414237 [==============================] - 1971s 5ms/step - loss: 0.0229 - accuracy: 0.9917
Epoch 6/6
414237/414237 [==============================] - 1983s 5ms/step - loss: 0.0211 - accuracy: 0.9921


In [12]:
pred = model.predict(test_x, verbose=1)
pred_y = np.argmax(pred, axis=1)

103560/103560 [==============================] - 208s 2ms/step


In [13]:
confusion_matrix(test_y, pred_y)

array([[ 117677,       0,      50,       0],
       [      0,   95279,    2923,       0],
       [  19026,     646, 2828238,       0],
       [      0,       0,       0,  250056]])

In [15]:
report=classification_report(test_y, pred_y, digits=5)
print(report)

              precision    recall  f1-score   support

           0    0.86082   0.99958   0.92502    117727
           1    0.99327   0.97023   0.98162     98202
           2    0.99895   0.99309   0.99601   2847910
           3    1.00000   1.00000   1.00000    250056

    accuracy                        0.99317   3313895
   macro avg    0.96326   0.99073   0.97566   3313895
weighted avg    0.99395   0.99317   0.99336   3313895

